<a href="https://colab.research.google.com/github/DannieHK/MachineLearning/blob/main/GPT2textPredicter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -O "sh.txt" "https://www.gutenberg.org/cache/epub/73328/pg73328.txt" -O THE_OUTERMOST_HOUSE.txt

--2024-04-11 12:12:57--  https://www.gutenberg.org/cache/epub/73328/pg73328.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 290554 (284K) [text/plain]
Saving to: ‘THE_OUTERMOST_HOUSE.txt’

THE_OUTERMOST_HOUSE 100%[===================>] 283.74K  1.22MB/s    in 0.2s    

2024-04-11 12:12:58 (1.22 MB/s) - ‘THE_OUTERMOST_HOUSE.txt’ saved [290554/290554]



In [2]:
with open("THE_OUTERMOST_HOUSE.txt", "r", encoding="utf-8") as file:
    text = file.read()

# Simple preprocessing: split the text into chunks if needed

In [ ]:
# Ensure necessary libraries are installed and updated
!pip install accelerate -U
!pip install transformers datasets -U

In [12]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained('gpt2')
model.config.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)

text_file = 'THE_OUTERMOST_HOUSE.txt'

# Create a dataset and a data collator for language modeling
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=text_file,
    block_size=128
) # Adjust based on your model and preferences

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # For causal (not masked) language modeling
)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',          # Where to store the training outputs
    overwrite_output_dir=True,
    num_train_epochs=5,              # Number of epochs (adjust as needed)
    per_device_train_batch_size=4,   # Batch size (adjust based on your GPU)
    save_steps=10_000,               # Save the model every 10,000 steps
    save_total_limit=2,              # Only keep the 2 most recent checkpoints
)

# Initialize Trainer.
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

trainer.train()

# Saving the trained model.
model.save_pretrained('./your_trained_model')
tokenizer.save_pretrained('./your_trained_model')


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Step,Training Loss
500,3.397800


('./your_trained_model/tokenizer_config.json',
 './your_trained_model/special_tokens_map.json',
 './your_trained_model/vocab.json',
 './your_trained_model/merges.txt',
 './your_trained_model/added_tokens.json')

In [13]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained('gpt2')
model.config.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)

In [14]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_path = './your_trained_model'  # Update this to your model's directory

tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)


In [15]:
# Prompt that will make it generate text with same structure as the book downloaded.
prompt = "East and ahead of the"

In [16]:
input_ids = tokenizer.encode(prompt, return_tensors='pt')

In [17]:
# Generate text
output_sequences = model.generate(
    input_ids=input_ids,
    max_length=100,  # Maximum length of the generated text
    num_return_sequences=1,  # Number of generated sequences
    temperature=0.7,  # Sampling temperature
    top_k=50,  # Only consider the top k tokens at each step
    top_p=0.9,  # Nucleus sampling
    repetition_penalty=1.2,  # Penalty for repetition
)

generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
print(generated_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


East and ahead of the
sea, I saw a great mass approaching from Cape Cod. The beach was full with surf; there were no fish in it at all! There had been an enormous wave that swept over this whole region on its way to

“the west coast last night? ” said Mr Abbott as he stood watching me watch--a huge one indeed!--and then paused for effectual seconds before returning again into his chair after ten oclock or so till eleven: _The
